# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 16 2022 8:30AM,246568,10,6971184,"Methapharm, Inc.",Released
1,Sep 16 2022 8:30AM,246569,10,SONSB0001924,"Nextsource Biotechnology, LLC",Released
2,Sep 16 2022 8:30AM,246569,10,SONSB0001939,"Nextsource Biotechnology, LLC",Released
3,Sep 16 2022 8:30AM,246573,10,0085984338,ISDIN Corporation,Released
4,Sep 16 2022 8:30AM,246573,10,0085984345,ISDIN Corporation,Released
5,Sep 16 2022 8:30AM,246573,10,0085984344,ISDIN Corporation,Released
6,Sep 16 2022 8:30AM,246573,10,0085984347,ISDIN Corporation,Released
7,Sep 16 2022 8:30AM,246573,10,0085984346,ISDIN Corporation,Released
8,Sep 16 2022 8:30AM,246573,10,0085984368,ISDIN Corporation,Released
9,Sep 16 2022 8:30AM,246573,10,0085984366,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,246580,Executing,6
16,246580,Released,2
17,246581,Released,5
18,246582,Released,2
19,246584,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246579,NaN,NaN,1.0
246580,NaN,6.0,2.0
246581,NaN,NaN,5.0
246582,NaN,NaN,2.0
246584,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246510,0.0,0.0,1.0
246517,0.0,0.0,1.0
246528,0.0,0.0,2.0
246532,1.0,0.0,15.0
246550,0.0,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246510,0,0,1
246517,0,0,1
246528,0,0,2
246532,1,0,15
246550,0,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246510,0,0,1
1,246517,0,0,1
2,246528,0,0,2
3,246532,1,0,15
4,246550,0,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246510,,,1
1,246517,,,1
2,246528,,,2
3,246532,1,,15
4,246550,,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc."
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC"
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation
11,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC"
12,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc."
13,Sep 16 2022 8:18AM,246582,19,"Graystone, LLC"
15,Sep 16 2022 8:14AM,246581,10,Emerginnova
20,Sep 16 2022 7:35AM,246580,20,Alumier Labs Inc.
28,Sep 16 2022 7:33AM,246579,19,"AdvaGen Pharma, Ltd"
29,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",,,1
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",,,2
2,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,,,8
3,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",,,1
4,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc.",,,1
5,Sep 16 2022 8:18AM,246582,19,"Graystone, LLC",,,2
6,Sep 16 2022 8:14AM,246581,10,Emerginnova,,,5
7,Sep 16 2022 7:35AM,246580,20,Alumier Labs Inc.,,6,2
8,Sep 16 2022 7:33AM,246579,19,"AdvaGen Pharma, Ltd",,,1
9,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,,
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,,
2,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,,
3,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc.",1,,
5,Sep 16 2022 8:18AM,246582,19,"Graystone, LLC",2,,
6,Sep 16 2022 8:14AM,246581,10,Emerginnova,5,,
7,Sep 16 2022 7:35AM,246580,20,Alumier Labs Inc.,2,6,
8,Sep 16 2022 7:33AM,246579,19,"AdvaGen Pharma, Ltd",1,,
9,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,,
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,,
2,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,,
3,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,NaN,NaN
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,NaN,NaN
2,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,NaN,NaN
3,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,NaN,NaN
4,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,0.0,0.0
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,0.0,0.0
2,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,0.0,0.0
3,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,0.0,0.0
4,Sep 16 2022 8:26AM,246584,19,"GCH Granules USA, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2465538,42,0.0,1.0
16,493095,3,0.0,0.0
19,986302,5,0.0,0.0
20,246580,2,6.0,0.0
21,246577,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2465538,42,0.0,1.0
1,16,493095,3,0.0,0.0
2,19,986302,5,0.0,0.0
3,20,246580,2,6.0,0.0
4,21,246577,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42,0.0,1.0
1,16,3,0.0,0.0
2,19,5,0.0,0.0
3,20,2,6.0,0.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,16,Released,3.0
2,19,Released,5.0
3,20,Released,2.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,6.0,0.0
Released,42.0,3.0,5.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,6.0,0.0
2,Released,42.0,3.0,5.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,6.0,0.0
2,Released,42.0,3.0,5.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()